### Importing data

In [2]:
import json

def read_jsonl_file(file_path):
    """
    read a JSONL file and return its contents as a list of dictionaries
    """
    with open(file_path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f]


train_data = read_jsonl_file("../data/train.jsonl")
dev_data = read_jsonl_file('../data/dev.jsonl')


### Data processings

In [4]:
def process_data(data):
    processed_data = []
    for entry in data:
        sentences = entry['sentences']
        ners = sorted(entry['ners'], key=lambda x: x[1] - x[0])
        modified_ners = sorted([(entity[0], entity[1] + 1, entity[2]) for entity in ners])
        processed_data.append((sentences, modified_ners))
    return processed_data

prepared_data = process_data(train_data)
prepared_data[0]

('Бостон взорвали Тамерлан и Джохар Царнаевы из Северного Кавказа\n\n19 апреля 2013 года в пригороде Бостона  проходит спецоперация по поимке 19-летнего Джохара Царнаева, подозреваемого в теракте на Бостонском марафоне 15 апреля и в смертельном ранении полицейского на кампусе Массачусетского технологического института 18 апреля.\n\nВторой подозреваемый, его брат, 26-летний Тамерлан Царнаев, был ранен в перестрелке в Уотертауне  и позже скончался в больнице.\n\nУотертаун и его окрестности фактически переведены на осадное положение: окрестности оцеплены, дороги перекрыты, магазины и бизнесы закрыты, жителей просят не выходить из домов и не приближаться к окнам, над районом спецоперации перекрыты полёты авиации.\n\nВ Бостоне приостановлена работа общественного транспорта, включая метро, автобусы, такси и пригородные поезда. Отменены занятия в Гарварде, Массачусетском технологическом институте, Университете Саффолка, Бостонском университете и во всех городских школах.\n\nНа сайте ФБР опубл

In [5]:
import re
from tqdm import tqdm

def convert_dataset(dataset):
    converted_dataset = []
    for sentence, entities in tqdm(dataset):
        tokens = re.findall(r'\b\w+\b', sentence.lower())  # use a regular expression to split into words
        tags = ['O'] * len(tokens)  # initialise the tags for each word as an 'O' (not an entity)
        for start, end, entity_type in entities:
            start_word_index = len(re.findall(r'\b\w+\b', sentence[:start].lower()))  # find the index of the beginning of the word
            end_word_index = len(re.findall(r'\b\w+\b', sentence[:end].lower())) - 1  # find the index of the end of the word
            if start_word_index == end_word_index:  # if the entity is within the same word
                # set tag to B-<entity> for the beginning of the entity
                tags[start_word_index] = 'B-' + entity_type
            else:
                tags[start_word_index] = 'B-' + entity_type
                for i in range(start_word_index + 1, end_word_index + 1):
                    tags[i] = 'I-' + entity_type # set tag to I-<entity> for the intermediate words
        converted_dataset.append((tokens, tags))
    return converted_dataset

prepared_data = convert_dataset(prepared_data)

100%|██████████| 519/519 [00:02<00:00, 188.74it/s]


In [6]:
from sklearn.model_selection import train_test_split

prepared_data_train, prepared_data_test = train_test_split(prepared_data, train_size=0.8)

In [7]:
prepared_data = {}
prepared_data['train'] = prepared_data_train
prepared_data['valid'], prepared_data['test'] =  train_test_split(prepared_data_test, test_size=0.5)

In [8]:
first_two_train_samples = prepared_data['train'][:2]
for n, sample in enumerate(first_two_train_samples):
    # sample is a tuple of sentence_tokens and sentence_tags
    tokens, tags = sample
    print('Sentence {}'.format(n))
    print('Tokens: {}'.format(tokens))
    print('Tags:   {}'.format(tags))

Sentence 0
Tokens: ['первое', 'поражение', 'серены', 'уильямс', 'в', 'финале', 'australian', 'open', 'серена', 'уильямс', '30', 'января', '2016', 'года', 'в', 'финале', 'открытого', 'чемпионата', 'австралии', 'первая', 'ракетка', 'мира', 'американка', 'серена', 'уильямс', 'проиграла', 'немке', 'ангелике', 'кербер', 'со', 'счётом', '6', '4', '3', '6', '6', '4', 'игра', 'шла', 'в', 'течении', 'двух', 'часов', 'и', 'восьми', 'минут', 'это', 'первый', 'турнир', 'большого', 'шлема', 'для', '28', 'летней', 'немки', 'серена', 'до', 'встречи', 'считалась', 'фаворитом', 'финала', 'так', 'как', 'она', 'уже', 'шесть', 'раз', 'проходил', 'в', 'финал', 'турнира', 'и', 'всё', 'время', 'его', 'выигрывала', 'если', 'бы', 'и', 'в', 'этот', 'раз', 'она', 'победила', 'то', 'она', 'бы', 'обновила', 'рекорд', 'за', 'всю', 'открытую', 'эру', 'напомним', 'что', 'открытая', 'эра', 'ведёт', 'отсчёт', 'с', '1968', 'года', 'когда', 'профессионалам', 'разрешили', 'принимать', 'участие', 'в', 'australian', 'open',

In [13]:
from ner.corpus import Corpus
corp = Corpus(prepared_data, embeddings_file_path=None) # all data is used for training

### Training model

In [17]:
from ner.network import NER
import tensorflow as tf

learning_params = {'dropout_rate': 0.5,
                'epochs': 15,
                'learning_rate': 0.005,
                'batch_size': 8,
                'learning_rate_decay': 0.707}

model_params = {"filter_width": 7,
            "embeddings_dropout": True,
            "n_filters": [
                128, 128,
            ],
            "token_embeddings_dim": 100,
            "char_embeddings_dim": 25,
            "use_batch_norm": True,
            "use_crf": True,
            "net_type": 'cnn',
            "use_capitalization": True,
            }

net = NER(corp, **model_params) # initialise the model
results = net.fit(**learning_params) # train the model

Number of parameters: 
Embeddings 2309400
ConvNet 228352
Classifier 7740
transitions:0 3600
Total number of parameters equal 2549092
Epoch 0
Eval on valid:
processed 11830 tokens with 2535 phrases; found: 1312 phrases; correct: 196.

precision:  14.94%; recall:  7.73%; FB1:  10.19


Epoch 1
Eval on valid:
processed 11830 tokens with 2535 phrases; found: 842 phrases; correct: 399.

precision:  47.39%; recall:  15.74%; FB1:  23.63


Epoch 2
Eval on valid:
processed 11830 tokens with 2535 phrases; found: 1419 phrases; correct: 741.

precision:  52.22%; recall:  29.23%; FB1:  37.48


Epoch 3
Eval on valid:
processed 11830 tokens with 2535 phrases; found: 1423 phrases; correct: 841.

precision:  59.10%; recall:  33.18%; FB1:  42.50


Epoch 4
Eval on valid:
processed 11830 tokens with 2535 phrases; found: 1574 phrases; correct: 961.

precision:  61.05%; recall:  37.91%; FB1:  46.78


Epoch 5
Eval on valid:
processed 11830 tokens with 2535 phrases; found: 1887 phrases; correct: 1086.

precisi

### Making predictions

In [26]:
def convert_back(tokens, tags):
    """
    convert tokens and tags back to a sentence and a list of entities
    """
    sentence = ""
    entities = []
    current_entity = None
    start_index = 0
    for token, tag in zip(tokens, tags):
        if tag.startswith("B-"):  # If this is the beginning of the entity
            if current_entity is not None:  # if there is an entity in progress
                entities.append([start_index, len(sentence.strip()), current_entity])
            current_entity = tag[2:]
            sentence += " " + token
            start_index = len(sentence.strip()) - len(token)
        elif tag.startswith("I-"):  # if this is the intermediate part of the entity
            if current_entity is not None:  # if there is an entity in progress
                sentence += " " + token
        else:  # if this is not an entity
            if current_entity is not None:  # if there is an entity in progress
                entities.append([start_index, len(sentence.strip()), current_entity])
                current_entity = None
            sentence += " " + token
    if current_entity is not None:  # if there is an entity in progress at the end of the sentence
        entities.append([start_index, len(sentence.strip()), current_entity])
    return entities

In [31]:
from ner.utils import tokenize, lemmatize

def save_to_jsonl(filename, data): # save the data to a JSONL file
    with open(filename, 'w', encoding='utf-8') as f:
        for entities in data:
            json_line = {"ners": entities}
            f.write(json.dumps(json_line, ensure_ascii=False) + '\n')

def predict(sentence, network):
    # split sentence into tokens
    tokens = tokenize(sentence)
    # lemmatize tokens
    tokens_lemmas = lemmatize(tokens)
    # predict tags
    tags = network.predict_for_token_batch([tokens_lemmas])[0]
    return tokens, tags

output = []
for sentence in tqdm(dev_data):
    tokens, tags = predict(sentence["senences"], net) # predict tags for the sentence
    output.append(convert_back(tokens, tags)) # convert tokens and tags back to entities

save_to_jsonl("test.jsonl", output)

100%|██████████| 65/65 [00:05<00:00, 10.93it/s]
